In [1]:
pip install SpeechRecognition

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install PyAudio

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import threading
import tkinter as tk
from tkinter import scrolledtext, messagebox
import speech_recognition as sr
from queue import Queue
from time import sleep
import datetime

class VoiceRecognitionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Voice Recognizer")
        self.root.geometry("600x600")
        
        # Colors for Eye Comfort
        self.bg_color = "#2b2b2b"  # Dark background
        self.text_color = "#f2f2f2"  # Light text for contrast
        self.highlight_color = "#b3ff66"  # Keyword highlight
        self.button_color = "#5f5f5f"
        self.button_text_color = "#ffffff"
        self.transcript_bg_color = "#3b3b3b"

        # Configure root window color
        self.root.configure(bg=self.bg_color)

        # Transcription Box
        self.transcription_box = scrolledtext.ScrolledText(root, 
                                                          width=60, 
                                                          height=20, 
                                                          wrap=tk.WORD, 
                                                          bg=self.transcript_bg_color, 
                                                          fg=self.text_color, 
                                                          insertbackground=self.text_color)
        self.transcription_box.pack(pady=10)

        # Start and Stop Buttons
        self.start_button = tk.Button(root, text="Start Recognition", command=self.start_recognition_thread,
                                      bg=self.button_color, fg=self.button_text_color)
        self.start_button.pack(pady=10)
        
        self.stop_button = tk.Button(root, text="Stop Recognition", command=self.stop_recognition,
                                     bg=self.button_color, fg=self.button_text_color)
        self.stop_button.pack(pady=10)

        # Incident Logging Button
        self.log_button = tk.Button(root, text="Log Incident", command=self.log_incident,
                                     bg=self.button_color, fg=self.button_text_color)
        self.log_button.pack(pady=10)

        # Keyword Detection Label
        self.keyword_label = tk.Label(root, text="Keyword Detection:", bg=self.bg_color, fg=self.text_color)
        self.keyword_label.pack(pady=5)

        # UI Configuration
        self.queue = Queue()
        self.is_running = threading.Event()
        self.recognition_thread = None

        # Speech recognizer
        self.recognizer = sr.Recognizer()
        self.microphone = sr.Microphone()

        # Adjust background noise in a separate thread
        self.background_noise_reduction()

        # Extended Keyword List for Criminal Detection
        self.keywords = [
            "thief", "steal", "robbery", "murder", "killer", "assault",
            "attack", "criminal", "suspect", "thug", "enemy", "threat", 
            "gun", "weapon", "danger", "emergency", "help"
        ]

    def background_noise_reduction(self):
        """Adjust for ambient noise."""
        with self.microphone as source:
            self.transcription_box.insert(tk.END, "Calibrating microphone for ambient noise, please wait...\n")
            self.recognizer.adjust_for_ambient_noise(source, duration=2)
            self.transcription_box.insert(tk.END, "Microphone calibrated. You can start speaking.\n")
            self.transcription_box.see(tk.END)

    def start_recognition_thread(self):
        """Start the voice recognition in a new thread."""
        if not self.recognition_thread or not self.recognition_thread.is_alive():
            self.is_running.set()
            self.recognition_thread = threading.Thread(target=self.voice_recognition)
            self.recognition_thread.start()
        else:
            messagebox.showinfo("Info", "Recognition is already running.")

    def stop_recognition(self):
        """Stop the voice recognition process."""
        if self.is_running.is_set():
            self.is_running.clear()
            self.transcription_box.insert(tk.END, "Recognition stopped.\n")
            self.transcription_box.see(tk.END)
        else:
            messagebox.showinfo("Info", "Recognition is not running.")

    def voice_recognition(self):
        """Threaded voice recognition process."""
        try:
            while self.is_running.is_set():
                with self.microphone as source:
                    self.transcription_box.insert(tk.END, "Listening...\n")
                    self.transcription_box.see(tk.END)
                    audio_data = self.recognizer.listen(source)
                
                try:
                    # Recognize the speech and add it to the queue
                    text = self.recognizer.recognize_google(audio_data)
                    self.queue.put(text)  # Add recognized text to queue

                    # Check for keywords in the recognized text
                    self.keyword_detection(text)

                except sr.UnknownValueError:
                    self.queue.put("Unrecognized speech\n")
                except sr.RequestError:
                    self.queue.put("API unavailable\n")
                
                # Update the transcription box with the latest text
                self.update_transcription_box()

        except Exception as e:
            print(f"Error during recognition: {e}")
            self.is_running.clear()

    def update_transcription_box(self):
        """Update the UI transcription box with recognized text."""
        try:
            while not self.queue.empty():
                recognized_text = self.queue.get()
                self.transcription_box.insert(tk.END, f"{recognized_text}\n")
                self.transcription_box.see(tk.END)  # Auto-scroll
        except tk.TclError as e:
            print(f"Error updating UI: {e}")

    def keyword_detection(self, text):
        """Check for keywords in the recognized text and highlight them."""
        for keyword in self.keywords:
            if keyword in text.lower():
                self.queue.put(f"Keyword Detected: {keyword.upper()}\n")
                self.transcription_box.insert(tk.END, f"Keyword Detected: {keyword.upper()}\n", 'highlight')
                self.transcription_box.tag_configure('highlight', foreground=self.highlight_color)
                self.transcription_box.see(tk.END)
                # Log the incident automatically
                self.log_incident()
                break

    def log_incident(self):
        """Log the detected incident with timestamp."""
        if self.queue.empty():
            return  # Nothing to log

        recognized_text = self.queue.get()
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log_entry = f"{timestamp} - {recognized_text.strip()}\n"
        
        with open("incident_log.txt", "a") as log_file:
            log_file.write(log_entry)
        
        self.transcription_box.insert(tk.END, "Incident logged.\n")
        self.transcription_box.see(tk.END)

if __name__ == "__main__":
    root = tk.Tk()
    app = VoiceRecognitionApp(root)
    root.mainloop()

import speech_recognition as sr

# Create a recognizer instance
recognizer = sr.Recognizer()

# Use the microphone as a source for input
with sr.Microphone() as source:
    print("Please say something:")
    # Adjust for ambient noise and record
    recognizer.adjust_for_ambient_noise(source)
    audio = recognizer.listen(source)

    print("Recognizing...")
    try:
        # Using Google Web Speech API to recognize the audio
        text = recognizer.recognize_google(audio)
        print("You said: " + text)
    except sr.UnknownValueError:
        print("Sorry, I could not understand the audio.")
    except sr.RequestError as e:
        print("Could not request results from Google Speech Recognition service; {0}".format(e))
